In [1]:
! pip install -q kaggle

In [45]:
from glob import glob
import numpy as np
import pandas as pd
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import Imagedata_generatorerator
from tensorflow.keras.layers import BatchNormalization, RandomFlip, RandomZoom, RandomRotation, Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten, Dropout, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

#from tqdm import tqdm_notebook,trange
print("done import")

done import


In [3]:
from google.colab import files

files.upload()

dataset = 'histopathologic-cancer-detection'

!rm -r $dataset

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c $dataset

zip_file = f"{dataset}.zip"
destination_dir = f"/content/{dataset}"

if not os.path.exists(zip_file):
    print(f"Error: {zip_file} not found.")
else:
    !unzip -q $zip_file -d $destination_dir
    !rm $zip_file

Saving kaggle.json to kaggle.json
rm: cannot remove 'histopathologic-cancer-detection': No such file or directory
rm: cannot remove '/root/.kaggle': No such file or directory
100% 6.30G/6.31G [00:51<00:00, 191MB/s]
100% 6.31G/6.31G [00:51<00:00, 133MB/s]


In [4]:
path = './histopathologic-cancer-detection/'
train_path = './histopathologic-cancer-detection/train/'
test_path = './histopathologic-cancer-detection/test/'

In [5]:
all_train_df = pd.read_csv(path+"train_labels.csv")
all_train_df.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [6]:
print("Train Size: {}".format(len(os.listdir(train_path))))
print("Test Size: {}".format(len(os.listdir(test_path))))

Train Size: 220025
Test Size: 57458


In [7]:
label_counts = all_train_df.label.value_counts()
print(label_counts)



label
0    130908
1     89117
Name: count, dtype: int64


In [49]:
all_train_df["id"] = all_train_df["id"].apply(lambda x: x + ".tif")
all_train_df["label"] = all_train_df["label"].astype(str)

In [60]:
data_generator = ImageDataGenerator(rescale=1./255.,
                            validation_split=0.2)

In [61]:
train_data_generator = data_generator.flow_from_dataframe(
    dataframe=all_train_df,
    directory=train_path,
    x_col="id",
    y_col="label",
    subset="training",
    batch_size=256,
    seed=13,
    class_mode="binary",
    target_size=(64,64),
    shuffle=True)

Found 176020 validated image filenames belonging to 2 classes.


In [62]:
val_data_generator = data_generator.flow_from_dataframe(
    dataframe=all_train_df,
    directory=train_path,
    x_col="id",
    y_col="label",
    subset="validation",
    batch_size=256,
    seed=13,
    class_mode="binary",
    target_size=(64,64),
    shuffle=True)

Found 44005 validated image filenames belonging to 2 classes.


In [64]:


model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(3,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(filters=16, kernel_size=(3,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(filters=32, kernel_size=(3,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(filters=32, kernel_size=(3,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(1, activation = "sigmoid"))

model.build(input_shape=(32, 64, 64, 3))

model.compile(Adam(0.001), loss = "binary_crossentropy", metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (32, 62, 62, 16)          448       
                                                                 
 batch_normalization_16 (Ba  (32, 62, 62, 16)          64        
 tchNormalization)                                               
                                                                 
 activation_8 (Activation)   (32, 62, 62, 16)          0         
                                                                 
 conv2d_19 (Conv2D)          (32, 60, 60, 16)          2320      
                                                                 
 batch_normalization_17 (Ba  (32, 60, 60, 16)          64        
 tchNormalization)                                               
                                                                 
 activation_9 (Activation)   (32, 60, 60, 16)         

In [ ]:
detection_cnn = model.fit(
                        train_data_generator,
                        steps_per_epoch=555,
                        epochs=7,
                        validation_data=val_data_generator,
                        validation_steps=185,
                        verbose=1)

Epoch 1/7
171/555 [========>.....................] - ETA: 1:07 - loss: 0.4303 - accuracy: 0.8076

In [58]:
model_acc = detection_cnn.history['accuracy']
val_model_acc = detection_cnn.history['val_accuracy']

epochs = range(len(model_acc))

# accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 3, 1)
plt.plot(epochs, model_acc, label='Train Accuracy')
plt.plot(epochs, val_model_acc, label='Val Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

NameError: name 'detection_cnn' is not defined